# Backcasting Demo Notebook

_Loren Champlin_

Adapted from _Adarsh Pyarelal_'s WM 12 Month Evaluation Notebook 

As always, we begin with imports, and print out the commit hash for a rendered
version of the notebook.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import numpy as np
import pandas as pd
from delphi.db import engine
import random as rm
import delphi.evaluation_port as EN
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger().setLevel(logging.CRITICAL)
from delphi.cpp.AnalysisGraph import AnalysisGraph as AG, InitialBeta as IB, RNG
import time
import seaborn as sns
import matplotlib.pyplot as plt

Here I will set random seeds

In [ ]:
np.random.seed(87)
rm.seed(87)

In [ ]:
G = AG.from_json_file("../scripts/data/raw/wm_12_month_evaluation/jsonld-merged20190404.json")

In [ ]:
G.print_indicators()
print("\n")

In [ ]:
H = G.get_subgraph_for_concept("UN/events/human/human_migration", depth=2, reverse=True)

In [ ]:
H.print_indicators()

In [ ]:
H.to_png()

In [ ]:
query = " ".join(
        [
            f"select * from indicator",
            f"where `Variable` like 'New asylum seeking applicants'",
        ]
    )

results = engine.execute(query)

df_results = pd.DataFrame(results,columns=results.keys())
df_results